In [11]:
import pandas as pd
import utils.fetcher_utils as fetcher
import utils.pipeline_util as pu
import utils.transformer_util as tu

In [12]:
# set pandas options
pd.set_option('display.max_columns', None)

In [13]:
# fetch the data 
df = fetcher.aquireIMDbDataFrame()
df.head()

Reading data from ../resources/movie_metadata.csv


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [14]:
# preprocess the data
# processed_df = preproc.preprocess_data(df)
# processed_df.head()
model, y_pred = pu.run_pipeline(df, True)

Training model
Creating train and test splits
Performing correlation analysis
                              Action  Adventure    Comedy     Crime     Drama  \
Action                      1.000000   0.337827 -0.223118  0.166321 -0.247493   
Adventure                   0.337827   1.000000 -0.024423 -0.171355 -0.265251   
Comedy                     -0.223118  -0.024423  1.000000 -0.076181 -0.260894   
Crime                       0.166321  -0.171355 -0.076181  1.000000  0.011285   
Drama                      -0.247493  -0.265251 -0.260894  0.011285  1.000000   
Family                     -0.071703   0.359228  0.240334 -0.141647 -0.212215   
Fantasy                     0.077305   0.307973  0.039342 -0.159523 -0.186287   
Horror                     -0.059171  -0.095093 -0.203710 -0.100261 -0.190570   
Romance                    -0.193077  -0.155303  0.232633 -0.128816  0.115193   
Sci-Fi                      0.347227   0.300841 -0.127747 -0.131882 -0.210812   
Thriller                    0.3

In [15]:
pd.Series(y_pred).describe()

count    666.000000
mean       6.327156
std        0.708905
min        4.919581
25%        5.823507
50%        6.261738
75%        6.669003
max       10.212718
dtype: float64